In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb  
import catboost as cbt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split


: 

In [ ]:
BASE_DIR = "../data/processed_data"

: 